<a href="https://colab.research.google.com/github/rohanjuneja/DNA-project/blob/main/deepconsensus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision.utils import save_image
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
import math
import os
import glob
import PIL
from PIL import Image
from torch.utils import data as D
from torch.utils.data.sampler import SubsetRandomSampler
import random
import pandas as pd
import time
import torch.autograd as autograd
from torch.autograd import Variable

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
# Positional Encoding
class PositionalEncoding(nn.Module):
    def __init__(self, dim_model, dropout_p, max_len):
        super().__init__()
        # Modified version from: https://pytorch.org/tutorials/beginner/transformer_tutorial.html
        # max_len determines how far the position can have an effect on a token (window)
        
        # Info
        self.dropout = nn.Dropout(dropout_p)
        
        # Encoding - From formula
        pos_encoding = torch.zeros(max_len, dim_model)
        positions_list = torch.arange(0, max_len, dtype=torch.float).view(-1, 1) # 0, 1, 2, 3, 4, 5
        division_term = torch.exp(torch.arange(0, dim_model, 2).float() * (-math.log(10000.0)) / dim_model) # 1000^(2i/dim_model)
        
        # PE(pos, 2i) = sin(pos/1000^(2i/dim_model))
        pos_encoding[:, 0::2] = torch.sin(positions_list * division_term)
        
        # PE(pos, 2i + 1) = cos(pos/1000^(2i/dim_model))
        pos_encoding[:, 1::2] = torch.cos(positions_list * division_term)
        
        # Saving buffer (same as parameter without gradients needed)
        pos_encoding = pos_encoding.unsqueeze(0).transpose(0, 1)
        self.register_buffer("pos_encoding",pos_encoding)
        
    def forward(self, token_embedding: torch.tensor) -> torch.tensor:
        # Residual connection + pos encoding
        return self.dropout(token_embedding + self.pos_encoding[:token_embedding.size(0), :])

In [ ]:
# Define the model
class DeepConsensus(nn.Module):
    
    # Constructor
    def __init__(self, num_tokens, dim_model, num_heads, num_encoder_layers, num_decoder_layers, dropout_p):
        super().__init__()

        # INFO
        self.model_type = "Transformer"
        self.dim_model = dim_model

        # LAYERS
        self.positional_encoder = PositionalEncoding(
            dim_model=dim_model, dropout_p=dropout_p, max_len=5000
        )
        self.embedding = nn.Embedding(num_tokens, dim_model)
        self.transformer = nn.Transformer(
            d_model=dim_model,
            nhead=num_heads,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dropout=dropout_p,
        )
        self.out = nn.Linear(dim_model, num_tokens)

    def forward(self, src, tgt):
        # Src size must be (batch_size, src sequence length)
        # Tgt size must be (batch_size, tgt sequence length)

        # Embedding + positional encoding - Out size = (batch_size, sequence length, dim_model)
        src = self.embedding(src) * math.sqrt(self.dim_model)
        tgt = self.embedding(tgt) * math.sqrt(self.dim_model)
        src = self.positional_encoder(src)
        tgt = self.positional_encoder(tgt)

        # Transformer blocks - Out size = (sequence length, batch_size, num_tokens)
        transformer_out = self.transformer(src, tgt)
        out = self.out(transformer_out)

        return out

In [ ]:
torch.set_default_tensor_type('torch.cuda.FloatTensor')

bmap = {"A":0, "C":1, "G":2, "T":3, "$":4}
def one_hot(b):
    t = [[0,0,0,0,0]]
    i = bmap[b]
    t[0][i] = 1
    return t

print("one-hot encoding for DNA bases")
print("A:", one_hot("A"))
print("C:", one_hot("C"))
print("G:", one_hot("G"))
print("T:", one_hot("T"))

one-hot encoding for DNA bases
A: [[1, 0, 0, 0, 0]]
C: [[0, 1, 0, 0, 0]]
G: [[0, 0, 1, 0, 0]]
T: [[0, 0, 0, 1, 0]]


In [ ]:
# Introducing the error in DNA sequence

def sim_error(seq, pi=0.05, pd=0.05, ps=0.01):
    """
    Given an input sequence `seq`, generating another
    sequence with errors. 
    pi: insertion error rate
    pd: deletion error rate
    ps: substitution error rate
    """
    out_seq = []
    for c in seq:
        while 1:
            r = random.uniform(0,1)
            if r < pi:
                out_seq.append(random.choice(["A","C","G","T"]))
            else:
                break
        r -= pi
        if r < pd:
            continue
        r -= pd
        if r < ps:
            out_seq.append(random.choice(["A","C","G","T"]))
            continue
        out_seq.append(c)
    return "".join(out_seq)

In [ ]:
seq = [random.choice(["A","C","G","T"]) for _ in range(120)]
print("".join(seq))

CGTTTAACTACGAGCCGGTAGATCCACTTTTTTTATGAAGATCTTTCGCGTAGCGGAGCATAGCTACATCATACGAAGTCCGGCAGTCCCTAAAATGGGGTGGGATTGCGTTAATTAGCT


In [ ]:
seq_vec = torch.FloatTensor([bmap[c] for c in seq])
print(seq_vec.shape)

torch.Size([120])


In [ ]:
print(seq_vec[0])

tensor(1., device='cpu')


### Generate Training data and Train the data 

In [ ]:
num_clusters = 10000
train_consensus_strands = []
train_target = []
for i in range(num_clusters):
    strand = [random.choice(["A","C","G","T"]) for _ in range(120)]
    train_consensus_strands.append(strand)
    strand_t = [bmap[c] for c in strand]
#     strand_t = Variable(torch.FloatTensor([one_hot(c) for c in strand]))
    train_target.append(strand_t)
print("".join(strand))

print(len(train_consensus_strands))
print(len(train_target))

train_target = torch.Tensor(train_target).long().cuda()

ATCGCACATAAGTGAATGCGGACCTGTTCCTCGTTGTGTACTTTATCGCAAAAGGTCGATATTTTCGACCACCGTCAGATTAGCACGTCACTTGCAGCTATTTTCTCTTTAAATACACGC
10000
10000


In [ ]:
print(train_target.shape)

torch.Size([10000, 120])


In [ ]:
train_data = []

for i in range(num_clusters):
    train_clusters = []
    seq = train_consensus_strands[i]
    for j in range(10):
        noisy_seq = sim_error(seq, pi=random.uniform(0.03, 0.09), pd=random.uniform(0.03, 0.09), 
        ps=random.uniform(0.03, 0.09))
        noisy_seq_t = [bmap[c] for c in noisy_seq]
#         noisy_seq_t = Variable(torch.FloatTensor([one_hot(c) for c in noisy_seq])).cuda()
        train_clusters.append(noisy_seq_t)
        
    train_data.append(train_clusters)

In [ ]:
max_length = 0
for i in range(num_clusters):
    for j in range(10):
        if (len(train_data[i][j]) > max_length):
            max_length = len(train_data[i][j])

print(max_length)

143


In [ ]:
min_length = 0
for i in range(num_clusters):
    for j in range(10):
        if (i == 0 and j == 0):
            min_length = len(train_data[i][j])
            
        elif (len(train_data[i][j]) < min_length):
            min_length = len(train_data[i][j])

print(min_length)

100


In [ ]:
for i in range(num_clusters):
    for j in range(10):
        diff = max_length - len(train_data[i][j])
        for k in range(diff):
            train_data[i][j].append(int(4))

train_data = torch.Tensor(train_data).long().cuda()

In [ ]:
print(train_data.shape)

torch.Size([10000, 10, 143])


In [ ]:
model = DeepConsensus(num_tokens=5, dim_model=120, num_heads=2, 
                      num_encoder_layers=6, num_decoder_layers=0, dropout_p=0.1)
model.cuda()
model.zero_grad()

In [ ]:
initial_lr = 0.1
lr=initial_lr
optimizer = optim.SGD(model.parameters(), lr=initial_lr)
criterion = nn.CrossEntropyLoss()

In [ ]:
def vec_to_one_hot(b):
    t = [[0,0,0,0,0]]
    i = b
    t[0][i] = 1
    return t

print("one-hot encoding for DNA bases")
print("A:", vec_to_one_hot(0))
print("C:", vec_to_one_hot(1))
print("G:", vec_to_one_hot(2))
print("T:", vec_to_one_hot(3))
print("$:", vec_to_one_hot(4))

one-hot encoding for DNA bases
A: [[1, 0, 0, 0, 0]]
C: [[0, 1, 0, 0, 0]]
G: [[0, 0, 1, 0, 0]]
T: [[0, 0, 0, 1, 0]]
$: [[0, 0, 0, 0, 1]]


In [ ]:
train_target_one_hot = []

for i in range(int(len(train_target))):
    train_target_one_hot.append([])
    for j in range(int(len(train_target[i]))):
        train_target_one_hot[i].append(vec_to_one_hot(int(train_target[i][j])))
        
train_target_one_hot = torch.Tensor(train_target_one_hot).long().cuda()
print(train_target_one_hot.shape)

torch.Size([10000, 120, 1, 5])


In [ ]:
# Training the model
num_epochs = 20

range_ = (1, 120)
# mini_batch_size = 10
for epoch in range(num_epochs):
    model.train()
    for i in range(int(len(train_data))):
        train_loss = 0
        s, e = range_
        optimizer.zero_grad()
        count = 0
        for seq in train_data[i]:
            model.zero_grad()
            count = count + 1
            # Noisy clusters (training data)
            seq = seq[s-1:e]
            seq_ = seq.view(-1,120)
            # Original string (training target)
            seq_target = train_target[i][s-1:e]
            seq_target = seq_target.view(-1, 120)
            # Forward Prop
            out = model(seq_, seq_target)
            # Loss computation
            train_loss += criterion(out, train_target_one_hot[i][count])
            
        # Backward propagation operation
        train_loss.backward()
        optimizer.step()
        
    print("Epoch:", epoch, "Training loss:", train_loss.cpu().item()/len(train_data), "learning rate:", lr)
        
    # Learning rate decay
    if epoch % 2 ==0:
        lr *= 0.95
        optimizer = optim.SGD(model.parameters(), lr=lr)
    
if (num_epochs > 0):
    torch.save(model.state_dict(), "deepconsensus.pt")

Epoch: 0 Training loss: 0.004787557220458984 learning rate: 0.1
Epoch: 1 Training loss: 0.00478189811706543 learning rate: 0.095
Epoch: 2 Training loss: 0.004785650253295899 learning rate: 0.095
Epoch: 3 Training loss: 0.004775938034057617 learning rate: 0.09025
Epoch: 4 Training loss: 0.004784066390991211 learning rate: 0.09025
Epoch: 5 Training loss: 0.004779855728149414 learning rate: 0.0857375
Epoch: 6 Training loss: 0.004784226989746094 learning rate: 0.0857375
Epoch: 7 Training loss: 0.004781884765625 learning rate: 0.08145062499999998
Epoch: 8 Training loss: 0.004780577850341797 learning rate: 0.08145062499999998
Epoch: 9 Training loss: 0.004782878494262695 learning rate: 0.07737809374999999
Epoch: 10 Training loss: 0.004785377502441407 learning rate: 0.07737809374999999
Epoch: 11 Training loss: 0.0047844989776611325 learning rate: 0.07350918906249998
Epoch: 12 Training loss: 0.0047859367370605465 learning rate: 0.07350918906249998
Epoch: 13 Training loss: 0.0047820453643798826 

In [ ]:
model.load_state_dict(torch.load("deepconsensus.pt"))

<All keys matched successfully>

### Generate Test Data 

In [ ]:
# DNA clusters to test model
num_test_clusters = 1000
test_consensus_strands = []
test_target = []

for i in range(num_test_clusters):
    strand = [random.choice(["A","C","G","T"]) for _ in range(120)]
    test_consensus_strands.append(strand)
    strand_t = [bmap[c] for c in strand]
    
    test_target.append(strand_t)
    
print("".join(strand))

print(len(test_consensus_strands))
print(len(test_target))

ATGCGCCCGCTAGATGGCATGGCAGTCTAAGAGGCCGCATAAGTACACAACTCGATCTGGTTACCGAACCGCCTTACCGCCTGCGAGTTACAGCTTTTTGCCGCATAGTCATCTCGCGGA
1000
1000


In [ ]:
test_target = torch.Tensor(test_target).long().cuda()

In [ ]:
test_data = []

for i in range(num_test_clusters):
    test_clusters = []
    seq = test_consensus_strands[i]
    for j in range(10):
        noisy_seq = sim_error(seq, pi=0.06, pd=0.06, ps=0.06)
        
        noisy_seq_t = [bmap[c] for c in noisy_seq]
        
        test_clusters.append(noisy_seq_t)
        
    test_data.append(test_clusters)

In [ ]:
max_length = 0
for i in range(num_test_clusters):
    for j in range(10):
        if (len(test_data[i][j]) > max_length):
            max_length = len(test_data[i][j])

print(max_length)

138


In [ ]:
min_length = 0
for i in range(num_test_clusters):
    for j in range(10):
        if (i == 0 and j == 0):
            min_length = len(test_data[i][j])
            
        elif (len(test_data[i][j]) < min_length):
            min_length = len(test_data[i][j])

print(min_length)

105


In [ ]:
for i in range(num_test_clusters):
    for j in range(10):
        diff = max_length - len(test_data[i][j])
        for k in range(diff):
            test_data[i][j].append(int(4))

test_data = torch.Tensor(test_data).long().cuda()

In [ ]:
print(test_data.shape)

torch.Size([1000, 10, 138])


In [ ]:
print(test_target.shape)

torch.Size([1000, 120])


In [ ]:
def one_hot_to_vec(one_hot):
    one_hot = np.array(one_hot)
    vec = np.argmax(one_hot, axis = 0)
    return vec

vec = torch.zeros(5).cpu()
vec[0] = 1
print(vec.shape)
print(vec.dtype)

print("Vectors for one hot of DNA bases")
print("A:", one_hot_to_vec(vec))
print("C:", one_hot_to_vec([0, 1, 0, 0, 0]))
print("G:", one_hot_to_vec([0, 0, 1, 0, 0]))
print("T:", one_hot_to_vec([0, 0, 0, 1, 0]))
print("$:", one_hot_to_vec([0, 0, 0, 0, 1]))

torch.Size([5])
torch.float32
Vectors for one hot of DNA bases
A: 0
C: 1
G: 2
T: 3
$: 4


In [ ]:
test_target_one_hot = []

for i in range(int(len(test_target))):
    test_target_one_hot.append([])
    for j in range(int(len(test_target[i]))):
        test_target_one_hot[i].append(vec_to_one_hot(int(test_target[i][j])))
        
test_target_one_hot = torch.Tensor(test_target_one_hot).long().cuda()
print(test_target_one_hot.shape)

torch.Size([1000, 120, 1, 5])


In [ ]:
range_ = (1, 120)
accuracy = 0

model.eval()
# Run again
for i in range(len(test_data)):
    test_loss = 0
    s, e = range_
    for seq in test_data[i]:
        start_time = time.time()
        model.zero_grad()
        # Original string (test target)
        seq_target = test_target[i][s-1:e]
        seq_target = seq_target.view(-1, 120)
        # Noisy clusters (test data)
        seq = seq[s-1:e]
        seq_ = seq.view(-1,120)
        out = model(seq_, seq_target)
        check_time = time.time()
        
        for j in range(120):
            if (one_hot_to_vec(out[0][j].detach().cpu().numpy()) == 
                one_hot_to_vec(test_target_one_hot[i][j].reshape(5).detach().cpu().numpy())):
                
                accuracy = accuracy + 1
                
        end_time = time.time()
        
print("Time for forward prop per DNA sequence = " + str(check_time - start_time) + " sec")
print("Time for accuracy calculation per DNA sequence = " + str(end_time - check_time) + " sec")
                
print("Accuracy = " + str(accuracy/(test_data.shape[0] * test_data.shape[1] * 120)))

Time for forward prop per DNA sequence = 0.006760358810424805 sec
Time for accuracy calculation per DNA sequence = 0.006854057312011719 sec
Accuracy = 0.75125
